In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
pyspark.__version__

'3.3.3'

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/06 15:18:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df_fhv = spark.read.option("header", "true").csv("data/raw/fhv/fhvhv_tripdata_2021-06.csv.gz")

In [8]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [9]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [10]:
import pandas as pd
df_pandas = pd.read_csv("data/raw/fhv/fhvhv_tripdata_2021-06.csv.gz", nrows=100)

In [11]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [12]:
#spark.createDataFrame(df_pandas).schema (if it has null values it will produce an error)
from pyspark.sql import types

schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [13]:
df_fhv_sc = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("data/raw/fhv/fhvhv_tripdata_2021-06.csv.gz")
    

In [23]:

df_fhv_sc \
    .repartition(12) \
    .write.parquet("data/pq/fhv/")

In [14]:
df_rep = spark.read \
    .parquet("data/pq/fhv/*")

In [15]:
df_rep.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02875|2021-06-16 22:08:45|2021-06-16 22:38:10|          48|         181|      N|                B02875|
|              B02875|2021-06-27 08:13:22|2021-06-27 08:16:18|          10|          10|      N|                B02875|
|              B02510|2021-06-13 22:34:33|2021-06-13 22:53:03|          89|         189|      N|                  null|
|              B02764|2021-06-15 11:15:06|2021-06-15 11:35:40|          36|          82|      N|                B02764|
|              B02510|2021-06-09 11:41:36|2021-06-09 11:45:32|         254|         254|      N|                  null|
|              B02510|2021-06-29 23:42:0

In [22]:
df_rep.createOrReplaceTempView("trips")

In [44]:
spark.sql(
"""
SELECT COUNT(*) AS trip_count
FROM trips
WHERE EXTRACT(DAY FROM pickup_datetime) = 15
""").show()

+----------+
|trip_count|
+----------+
|    452470|
+----------+



In [19]:
from pyspark.sql import functions as F

# ceate a timestamp literal to compare with
filter_date = F.lit('2021-06-15').cast("timestamp") # casted to timestamp will  be 2021-06-15 00:00:00
comp = F.lit('2021-06-15').cast("timestamp")

filtered_df = df_rep.filter((F.col("pickup_datetime") >= filter_date) & (F.col("pickup_datetime") < filter_date))
filtered_df.show()


+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|dispatching_base_num|pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+
+--------------------+---------------+----------------+------------+------------+-------+----------------------+



In [49]:
total = filtered_df.count()
print(total)

2


In [30]:
df_duration = spark.sql("""
WITH cte AS (
  SELECT 
    dispatching_base_num, 
    (unix_timestamp(dropoff_datetime)) - (unix_timestamp(pickup_datetime)) / 3600 AS hrs
  FROM trips
)
SELECT MAX(hrs) AS mx
FROM cte;
""")

In [31]:
df_duration.show()

+--------------------+
|                  mx|
+--------------------+
|1.6246425001691666E9|
+--------------------+



In [65]:
df_zones = spark.read.parquet('zones/')

In [66]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [67]:
df_fhv_zone = df_rep.join(df_zones, df_rep.PULocationID == df_zones.LocationID)

In [69]:
df_fhv_zone.drop('LocationID').show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+--------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|  Borough|                Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+--------------------+------------+
|              B02875|2021-06-16 22:08:45|2021-06-16 22:38:10|          48|         181|      N|                B02875|Manhattan|        Clinton East| Yellow Zone|
|              B02875|2021-06-27 08:13:22|2021-06-27 08:16:18|          10|          10|      N|                B02875|   Queens|        Baisley Park|   Boro Zone|
|              B02510|2021-06-13 22:34:33|2021-06-13 22:53:03|          89|         189|      N|                  null| Brooklyn|Flatbush/Ditmas Park|   Boro Zone|
|              B

In [71]:
df_id_zone = df_fhv_zone.select('PULocationID','Zone' )

In [73]:
df_id_zone.show()

+------------+--------------------+
|PULocationID|                Zone|
+------------+--------------------+
|          48|        Clinton East|
|          10|        Baisley Park|
|          89|Flatbush/Ditmas Park|
|          36|      Bushwick North|
|         254|Williamsbridge/Ol...|
|         148|     Lower East Side|
|         243|Washington Height...|
|          72|East Flatbush/Rem...|
|         262|      Yorkville East|
|          51|          Co-Op City|
|         212|  Soundview/Bruckner|
|         226|           Sunnyside|
|         135|   Kew Gardens Hills|
|          50|        Clinton West|
|         174|             Norwood|
|         132|         JFK Airport|
|          52|         Cobble Hill|
|         116|    Hamilton Heights|
|          17|             Bedford|
|          86|        Far Rockaway|
+------------+--------------------+
only showing top 20 rows



In [72]:
df_id_zone.createOrReplaceTempView("pick_zones")

In [79]:
spark.sql("""
WITH cte AS (
SELECT PULocationID, Zone, COUNT(*) AS times
FROM pick_zones
GROUP BY PULocationID, Zone 
)
SELECT Zone
FROM cte
ORDER BY times DESC
LIMIT 1
""").show()

+-------------------+
|               Zone|
+-------------------+
|Crown Heights North|
+-------------------+

